# Web Scraping 

Neste notebook iremos apresentar um rápido projeto de web scraping. É um projeto inicial, a ideia é criar um primeiro contato com a área.

## O que é web scraping?

Em poucas palavras: `é a extração de informação de um site.` Essa informação é acessada por meio das informações contidas no `html` da página, assim, pode-se extrair imagens, valores (`dados em geral`) que são utilizados, principalmente, na análise dos dados.

A principal ideia do `web scraping` é automatizar uma tarefa que normalmente é feita pelo olhar humano, sendo possível coletar informações sem `gastar energia` e em pouco tempo. Ou seja, economizando tempo e esforço.

## É necessário ter cuidados

Sim, é necessário ter cuidados, já que mesmo sendo uma prática legal - sob a ótica da lei -, existem sites com políticas que não permitem tal extração de dados. Alguns cuidados interessantes:

- visualizar o arquivo `robots.txt`: nele está contido as restrições impostas. 
- termos de serviço.
- leis locais em que o site está hospedado.
- taxas de rastreamento.
- criar um arquivo de identificação do Scraper.
- proteger os dados coletados.


# Importações

Nesse projeto utilizaremos as seguintes bibliotecas em python:

- [urllib.request](https://docs.python.org/3/library/urllib.request.html) : é um módulo com diversas funções auxiliares que nos ajudam a acessar URLs.
- [bs4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) : é uma biblioteca voltada para extração de dados (web scraping).
- [pandas](https://pandas.pydata.org/docs/user_guide/index.html) : é uma bibliteca voltada para manipulação e análise de dados.

In [1]:
# Importando bibliotecas
from urllib.request import urlopen, urlretrieve
from bs4 import BeautifulSoup
import pandas as pd

# A rotina de web scraping

Aqui iremos descrever passo a passo - e tentando explicar -  a rotina aplicada para extrair os dados de determinado site.

## Obtendo a quantidade de páginas

O site em questão possui um dashboard com páginas, o primeiro passo vai ser descobrir quantas páginas esse dashboard possui.

In [2]:
def totalDePaginas(url: str) -> int:
    """
        Irá buscar a informação do total de páginas que existem no dashboard.
        Retorna a quantidade de páginas.
        
        url: string que contém a url do site.
    """
    # Criando o response com a url
    response = urlopen(url)
    # Obtendo o html da página
    html = response.read().decode('utf-8')
    # Criando o objeto BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')
    # Obtendo a quantidade de páginas por meio do método find e análise da classe
    pages = int(soup.find('span', class_="info-pages").get_text().split()[-1])
    
    # Retorna a quantidade de páginas encontradas
    return pages

pages = totalDePaginas('https://alura-site-scraping.herokuapp.com/index.php');
pages

25

## Obtendo os anúncios de cada página

Sabendo quantas páginas possui, agora queremos obter os anúncios de cada página.

In [3]:
def obtendoAnuncioPagina(numPag: int) -> BeautifulSoup:
    """
        Irá buscar os anúncios de uma página do dashboard.
        Retorna um objeto contendo os anuncios.
        
        numPag: o número da página em questão.
    """
    
    # Criando o response com a url
    response = urlopen('https://alura-site-scraping.herokuapp.com/index.php?page=' + str(numPag + 1))
    # Obtendo o html da página
    html = response.read().decode('utf-8')
    # Criando o objeto BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Obtendo as TAGs de interesse
    anuncios = soup.find('div', {"id": "container-cards"}).findAll('div', class_="card")
    
    return anuncios

anuncios = obtendoAnuncioPagina(2)
anuncios

[<div class="well card">
 <div class="col-md-3 image-card">
 <img alt="Foto" height="155" src="https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/porsche-911-gt3rs/porsche-911-gt3rs-2933986__340.jpg" width="220"/>
 </div>
 <div class="col-md-6 body-card">
 <p class="txt-name inline">PORSCHE 911 GT3RS</p>
 <p class="txt-category badge badge-secondary inline">USADO</p>
 <p class="txt-motor">Motor Diesel</p>
 <p class="txt-description">Ano 1991 - 28.617 km</p>
 <ul class="lst-items">
 <li class="txt-items">► Painel digital</li>
 <li class="txt-items">► Controle de estabilidade</li>
 <li class="txt-items">► 4 X 4</li>
 <li class="txt-items">► Sensor de chuva</li>
 <li class="txt-items">...</li>
 </ul>
 <p class="txt-location">Belo Horizonte - MG</p>
 </div>
 <div class="col-md-3 value-card">
 <div class="value">
 <p class="txt-value">R$ 285.000</p>
 </div>
 </div>
 </div>,
 <div class="well card">
 <div class="col-md-3 image-card">
 <img alt="Foto" height="155" src="ht

## Obtendo as informações de cada anúncio

Tendo criado os anúncios, agora podemos extrair as informações desejadas de cada anúncio.

In [4]:
def obtendoInfosAnuncio(card: dict, anuncio: BeautifulSoup, needsImage: bool = False) -> dict:
    """
        Irá buscar as informações presente no anúncio em questão.
        Retorna um dicionário contendo todas as informações.
        
        card: um dicionário contendo a informação de um anúncio.
        anuncio: objeto que representa um anúncio do site.
    """
    
    # Obtenho o valor do anuncio
    card['value'] = anuncio.find('p', {'class': 'txt-value'}).getText()

    # Obtenho demais informações
    infos = anuncio.find('div', {'class': 'body-card'}).findAll('p')
    for info in infos:
        card[info.get('class')[0].split('-')[-1]] = info.get_text()

    # Obtenho mais informações - Acessórios
    items = anuncio.find('div', {'class': 'body-card'}).ul.findAll('li')
    items.pop()
    acessorios = []
    for item in items:
        acessorios.append(item.get_text().replace('► ', ''))
    card['items'] = acessorios
    
    if needsImage:
        # Busca as imagens do anúncio e salva como output
        image = anuncio.find('div', {'class': 'image-card'}).img
        urlretrieve(image.get('src'), './output/img/' + image.get('src').split('/')[-1]) 
    
    return card

card = {}
card = obtendoInfosAnuncio(card, anuncios[0])
card

{'value': 'R$ 285.000',
 'name': 'PORSCHE 911 GT3RS',
 'category': 'USADO',
 'motor': 'Motor Diesel',
 'description': 'Ano 1991 - 28.617 km',
 'location': 'Belo Horizonte - MG',
 'items': ['Painel digital',
  'Controle de estabilidade',
  '4 X 4',
  'Sensor de chuva']}

## Juntando cada passo...

Agora vamos demonstrar a rotina como um todo.

In [5]:
def scraper() -> pd.DataFrame:
    cards = []
    
    pages = totalDePaginas('https://alura-site-scraping.herokuapp.com/index.php');
    
    for page in range(pages):
        
        anuncios = obtendoAnuncioPagina(page)
        
        for anuncio in anuncios:
            
            card = {}
            
            cards.append(obtendoInfosAnuncio(card, anuncio))
        
     
    cards_df = pd.DataFrame(cards)
    return cards_df

cards = scraper()
cards

,value,name,category,motor,description,location,items,opportunity
0,R$ 338.000,LAMBORGHINI AVENTADOR,USADO,Motor 1.8 16v,Ano 1993 - 55.286 km,Belo Horizonte - MG,"[4 X 4, Câmera de estacionamento, Controle de ...",NaN
1,R$ 346.000,BMW M2,USADO,Motor 3.0 32v,Ano 2018 - 83.447 km,Belo Horizonte - MG,"[Câmera de estacionamento, Controle de estabil...",NaN
2,R$ 480.000,ALFA,USADO,Motor 1.8 16v,Ano 2004 - 19.722 km,Rio de Janeiro - RJ,"[Central multimídia, Bancos de couro, Rodas de...",NaN
3,R$ 133.000,PUECH,USADO,Motor Diesel V8,Ano 1992 - 34.335 km,São Paulo - SP,"[Bancos de couro, Freios ABS, Rodas de liga, C...",NaN
4,R$ 175.000,LAMBORGHINI MURCIELAGO,USADO,Motor 1.0 8v,Ano 1991 - 464 km,Belo Horizonte - MG,"[Central multimídia, Teto panorâmico, Sensor c...",NaN
...,...,...,...,...,...,...,...,...
241,R$ 489.000,SUV REAR TIRE,USADO,Motor 3.0 32v,Ano 1998 - 74.292 km,São Paulo - SP,"[Câmera de estacionamento, Rodas de liga, Sens...",NaN
242,R$ 427.000,ANTIQUE,NOVO,Motor 2.0 16v,Ano 2019 - 0 km,Belo Horizonte - MG,"[Bancos de couro, Freios ABS, Sensor de estaci...",NaN
243,R$ 203.000,SPORT,USADO,Motor 2.0 16v,Ano 2001 - 102.776 km,Belo Horizonte - MG,"[Sensor crepuscular, Sensor de chuva, Vidros e...",NaN
244,R$ 474.000,IMPERIAL,USADO,Motor 1.8 16v,Ano 2011 - 101.787 km,Belo Horizonte - MG,"[Painel digital, Travas elétricas, Sensor de c...",OPORTUNIDADE
